# Facial Detection

The goal of this python file is to fully train a model to recognize the age, gender, and emotion of a face that it detects either from an uplaoded image or live camera detection

#### 1. Imports and basic configuration

In [10]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)

# Image + training settings 
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS_HEADS = 10
EPOCHS_FINETUNE = 10

TensorFlow version: 2.20.0


#### 2. Load merged CSV and compute class counts

In [11]:
csv_path = "data/merged_dataset.csv"
df = pd.read_csv(csv_path)       # columns: image_path, age, gender, emotion [file:1]

print(df.head())

# Analyze the distribution of age, gender, and emotion labels/ class distributions
print("\nAge value counts:\n", df["age"].value_counts())
print("\nGender value counts:\n", df["gender"].value_counts())
print("\nEmotion value counts:\n", df["emotion"].value_counts())

                                          image_path  age  gender  emotion
0  source_data/UTK-Face/part3/27_0_1_201701201338...    2       0       -1
1  source_data/UTK-Face/part3/24_0_3_201701191655...    2       0       -1
2  source_data/UTK-Face/part3/8_1_0_2017011715460...    0       1       -1
3  source_data/UTK-Face/part3/85_1_0_201701202226...    6       1       -1
4  source_data/UTK-Face/part3/26_1_0_201701191929...    2       1       -1

Age value counts:
 age
 2    12339
-1     5102
 0     4823
 3     4754
 6     2936
 4     2460
 5     2395
 1     1564
Name: count, dtype: int64

Gender value counts:
 gender
0    20347
1    16026
Name: count, dtype: int64

Emotion value counts:
 emotion
-1    24102
 4     4772
 7     2524
 5     1982
 1     1290
 3      717
 6      705
 2      281
Name: count, dtype: int64


In [12]:
# If image paths in CSV are relative, prepend a root dir
root_dir = "/Users/renubandaru/Code/GitHub/facial-profiler/data/"  # change this if images are under a specific folder

print("\nOriginal image paths:\n", df["image_path"].head())
df["image_path"] = df["image_path"].apply(lambda p: os.path.join(root_dir, p))

print("\nUpdated image paths:\n")
print(df["image_path"].head())

# print("Before:\n", df["image_path"].head())        # before applying join
# df["image_path"] = df["image_path"].apply(lambda p: os.path.join(root_dir, p))
# print("After:\n", df["image_path"].head())         # after applying join


# Compute number of classes
NUM_AGE_CLASSES = df["age"].max() + 1          # ages are binned 0..6 [file:1]
NUM_GENDER_CLASSES = df["gender"].max() + 1    # usually 2 [file:1]
valid_emotions = df[df["emotion"] >= 0]["emotion"].unique()
NUM_EMOTION_CLASSES = int(valid_emotions.max() + 1)

print("NUM_AGE_CLASSES:", NUM_AGE_CLASSES)
print("NUM_GENDER_CLASSES:", NUM_GENDER_CLASSES)
print("NUM_EMOTION_CLASSES:", NUM_EMOTION_CLASSES)


Original image paths:
 0    source_data/UTK-Face/part3/27_0_1_201701201338...
1    source_data/UTK-Face/part3/24_0_3_201701191655...
2    source_data/UTK-Face/part3/8_1_0_2017011715460...
3    source_data/UTK-Face/part3/85_1_0_201701202226...
4    source_data/UTK-Face/part3/26_1_0_201701191929...
Name: image_path, dtype: object

Updated image paths:

0    /Users/renubandaru/Code/GitHub/facial-profiler...
1    /Users/renubandaru/Code/GitHub/facial-profiler...
2    /Users/renubandaru/Code/GitHub/facial-profiler...
3    /Users/renubandaru/Code/GitHub/facial-profiler...
4    /Users/renubandaru/Code/GitHub/facial-profiler...
Name: image_path, dtype: object
NUM_AGE_CLASSES: 7
NUM_GENDER_CLASSES: 2
NUM_EMOTION_CLASSES: 8


#### Train and Validation Split

In [13]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

print(f"Training samples (80%): {len(train_df)}, Validation samples (20%): {len(val_df)}")

Training samples (80%): 29098, Validation samples (20%): 7275


#### 4. Building tf.data pipelines with masking

Key idea: each sample returns image + labels + masks so losses can ignore missing emotion.

In [14]:
def load_and_preprocess(image_path):
    #Read an image from a file, decode it into a dense tensor, and normalize for EfficientNet.
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Ensure 3 color channels
    img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = tf.cast(img, tf.float32)
    img = keras.applications.efficientnet.preprocess_input(img)
    return img

def make_dataset(df_in, shuffle=True):
    # Create a tf.data.Dataset from the DataFrame that yields (image, labels_dict) with masks.
    image_paths = df_in["image_path"].values.astype("str")
    ages = df_in["age"].values.astype("int32")
    genders = df_in["gender"].values.astype("int32")
    emotions = df_in["emotion"].values.astype("int32")

    # Create a tf.data.Dataset from the image paths and labels
    ds = tf.data.Dataset.from_tensor_slices((image_paths, ages, genders, emotions))

    def _map_fn(image_path, age, gender, emotion):
        img = load_and_preprocess(image_path)

        # Build masks for each task: 1 if label is valid, 0 if -1 (invalid)
        has_age = tf.cast(age >= 0, tf.float32)
        has_gender = tf.cast(gender >= 0, tf.float32)
        has_emotion = tf.cast(emotion >= 0, tf.float32)

        # Replace invalid labels with 0 (or any dummy value) since they won't contribute to loss
        age = tf.where(age >= 0, age, 0)
        gender = tf.where(gender >= 0, gender, 0)
        emotion = tf.where(emotion >= 0, emotion, 0)

        labels = {
            "age": age,
            "gender": gender,
            "emotion": emotion,
            "has_age": has_age,
            "has_gender": has_gender,
            "has_emotion": has_emotion
        }

        return img, labels
    
    ds = ds.map(_map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(df_in), 2048), reshuffle_each_iteration=True)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_df, shuffle=True)
val_ds = make_dataset(val_df, shuffle=False)

# quick sanity check
batch_imgs, batch_labels = next(iter(train_ds))
print("Batch images:", batch_imgs.shape)
print("Label keys:", batch_labels.keys())


Batch images: (32, 224, 224, 3)
Label keys: dict_keys(['age', 'gender', 'emotion', 'has_age', 'has_gender', 'has_emotion'])


#### 5. EfficientNetB0 multi-task model

In [15]:
# Define EfficientNetB0 multitask model
def build_efficientnet_multitask(num_age, num_gender, num_emotion):

    # Input layer 
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name = "image")

    # EfficientNetB0 backbone - Base model with pretrained ImageNet weights, excluding top layers
    base = keras.applications.EfficientNetB0(
        include_top=False, 
        weights="imagenet", 
        input_tensor=inputs, 
        pooling="avg"           # global average pooling to get a single vector per image
    )

    # Start with backbone frozen for head training
    base.trainable = False

    # Shared dense layer for all tasks
    x = base.output
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    # Age head (7 classes)
    age_output = layers.Dense(num_age, activation="softmax", name="age_logits")(x)
    
    # Gender head
    gender_output = layers.Dense(num_gender, activation="softmax", name="gender_logits")(x)

    # Emotion head
    emotion_output = layers.Dense(num_emotion, activation="softmax", name="emotion_logits")(x)

    # Build the model
    model = keras.Model(
        inputs=inputs, 
        outputs=[age_output, gender_output, emotion_output],
        name="multi_task_efficientnet",
    )

    return model, base


model, base_model = build_efficientnet_multitask(
    NUM_AGE_CLASSES, NUM_GENDER_CLASSES, NUM_EMOTION_CLASSES
)
model.summary()


Model: "multi_task_efficientnet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ image[0][0]       │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,381,876 (16.72 MB)

 Trainable params: 332,305 (1.27 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

#### 6. Loss objects and custom Model sub class

In [16]:
# Per-sample sparse categorical crossentropy loss with masking for invalid labels(no reduction)
ce_age = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)
ce_gender = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)
ce_emotion = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)

# Task weights (you can tune these)
W_AGE = 1.0
W_GENDER = 1.0
W_EMOTION = 1.0

In [24]:
# Custom model to handle masked losses and metrics for multi-task learning

class MultiTaskModel(keras.Model):
    """Custom model that applies masked losses for multi-task learning."""
    def __init__(self, core_model, **kwargs):
        super().__init__(**kwargs)
        self.core_model = core_model
        

        # Accuracy metrics
        self.age_acc = keras.metrics.SparseCategoricalAccuracy(name="age_acc")
        self.gender_acc = keras.metrics.SparseCategoricalAccuracy(name="gender_acc")
        self.emotion_acc = keras.metrics.SparseCategoricalAccuracy(name="emotion_acc")

    @property
    def metrics(self):
        return [self.age_acc, self.gender_acc, self.emotion_acc]

    
    def train_step(self, data):
        images, labels = data

        age_true = labels["age"]
        gender_true = labels["gender"]
        emotion_true = labels["emotion"]

        has_age = labels["has_age"]
        has_gender = labels["has_gender"]
        has_emotion = labels["has_emotion"]

        with tf.GradientTape() as tape:
                age_logit, gender_logit, emotion_logit = self.core_model(images, training=True)

                age_loss = ce_age(age_true, age_logit) * has_age
                gender_loss = ce_gender(gender_true, gender_logit) * has_gender
                emotion_loss = ce_emotion(emotion_true, emotion_logit) * has_emotion

                eps = 1e-6
                age_loss = tf.reduce_sum(age_loss) / (tf.reduce_sum(has_age) + eps)
                gender_loss = tf.reduce_sum(gender_loss) / (tf.reduce_sum(has_gender) + eps)
                emotion_loss = tf.reduce_sum(emotion_loss) / (tf.reduce_sum(has_emotion) + eps)

                total_loss = W_AGE*age_loss + W_GENDER*gender_loss + W_EMOTION*emotion_loss

        grads = tape.gradient(total_loss, self.core_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.core_model.trainable_variables))

        self.age_acc.update_state(age_true, age_logit, sample_weight=has_age)
        self.gender_acc.update_state(gender_true, gender_logit, sample_weight=has_gender)
        self.emotion_acc.update_state(emotion_true, emotion_logit, sample_weight=has_emotion)

        return {
            "loss": total_loss,
            "age_loss": age_loss,
            "gender_loss": gender_loss,
            "emotion_loss": emotion_loss,
            "age_acc": self.age_acc.result(),
            "gender_acc": self.gender_acc.result(),
            "emotion_acc": self.emotion_acc.result(),
        }

    
    def test_step(self, data):
        images, labels = data

        age_true = labels["age"]
        gender_true = labels["gender"]
        emotion_true = labels["emotion"]

        has_age = labels["has_age"]
        has_gender = labels["has_gender"]
        has_emotion = labels["has_emotion"]

        age_logit, gender_logit, emotion_logit = self.core_model(
            images, training=False
        )

        age_loss = ce_age(age_true, age_logit) * has_age
        gender_loss = ce_gender(gender_true, gender_logit) * has_gender
        emotion_loss = ce_emotion(emotion_true, emotion_logit) * has_emotion

        eps = 1e-6
        age_loss = tf.reduce_sum(age_loss) / (tf.reduce_sum(has_age) + eps)
        gender_loss = tf.reduce_sum(gender_loss) / (tf.reduce_sum(has_gender) + eps)
        emotion_loss = tf.reduce_sum(emotion_loss) / (tf.reduce_sum(has_emotion) + eps)

        total_loss = (W_AGE * age_loss +
                      W_GENDER * gender_loss +
                      W_EMOTION * emotion_loss)

        # Update metrics
        self.age_acc.update_state(age_true, age_logit, sample_weight=has_age)
        self.gender_acc.update_state(gender_true, gender_logit, sample_weight=has_gender)
        self.emotion_acc.update_state(emotion_true, emotion_logit, sample_weight=has_emotion)

        return {
            "loss": total_loss,
            "age_loss": age_loss,
            "gender_loss": gender_loss,
            "emotion_loss": emotion_loss,
            "age_acc": self.age_acc.result(),
            "gender_acc": self.gender_acc.result(),
            "emotion_acc": self.emotion_acc.result(),
        }

#### 7. Phase 1 - Training the heads with frozen EfficientNetB0

This stage trains the shared dense + three heads while keeping EfficientNetB0 fixed.

In [25]:
# Phase 1 - Train only the heads with backbone frozen


multi_task_model = MultiTaskModel(core_model=model)

multi_task_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3)
)

history_heads = multi_task_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEADS
)


Epoch 1/10
138/910 ━━━━━━━━━━━━━━━━━━━━ 3:08 244ms/step - age_acc: 0.5146 - age_loss: 1.2392 - emotion_acc: 0.3730 - emotion_loss: 1.5522 - gender_acc: 0.7590 - gender_loss: 0.4357 - loss: 3.2271

Corrupt JPEG data: premature end of data segment


310/910 ━━━━━━━━━━━━━━━━━━━━ 2:31 252ms/step - age_acc: 0.5379 - age_loss: 1.1732 - emotion_acc: 0.4183 - emotion_loss: 1.4603 - gender_acc: 0.7811 - gender_loss: 0.4188 - loss: 3.0523

Corrupt JPEG data: premature end of data segment


910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - age_acc: 0.5605 - age_loss: 1.1066 - emotion_acc: 0.4713 - emotion_loss: 1.3330 - gender_acc: 0.8038 - gender_loss: 0.3859 - loss: 2.8255

...

310/910 ━━━━━━━━━━━━━━━━━━━━ 4:46 478ms/step - age_acc: 0.6975 - age_loss: 0.7616 - emotion_acc: 0.7290 - emotion_loss: 0.7578 - gender_acc: 0.8705 - gender_loss: 0.2892 - loss: 1.8086

Corrupt JPEG data: premature end of data segment


910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - age_acc: 0.7010 - age_loss: 0.7676 - emotion_acc: 0.7247 - emotion_loss: 0.7677 - gender_acc: 0.8731 - gender_loss: 0.2779 - loss: 1.8132

2026-02-22 20:00:09.882176: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


910/910 ━━━━━━━━━━━━━━━━━━━━ 578s 633ms/step - age_acc: 0.7010 - age_loss: 0.7673 - emotion_acc: 0.7247 - emotion_loss: 0.7673 - gender_acc: 0.8731 - gender_loss: 0.2781 - loss: 1.8128 - val_age_acc: 0.6321 - val_age_loss: 1.0662 - val_emotion_acc: 0.6570 - val_emotion_loss: 1.1628 - val_gender_acc: 0.8557 - val_gender_loss: 0.2050 - val_loss: 2.4340


In [26]:
# Print accuracies from Phase 1
print("Phase 1 - Head Training Results:")
print(f"Age Accuracy: {history_heads.history['age_acc'][-1]:.4f}")
print(f"Gender Accuracy: {history_heads.history['gender_acc'][-1]:.4f}")
print(f"Emotion Accuracy: {history_heads.history['emotion_acc'][-1]:.4f}")

print("\nPhase 1 - Validation Results:")
print(f"Val Age Accuracy: {history_heads.history['val_age_acc'][-1]:.4f}")
print(f"Val Gender Accuracy: {history_heads.history['val_gender_acc'][-1]:.4f}")
print(f"Val Emotion Accuracy: {history_heads.history['val_emotion_acc'][-1]:.4f}")

Phase 1 - Head Training Results:
Age Accuracy: 0.7015
Gender Accuracy: 0.8772
Emotion Accuracy: 0.7240

Phase 1 - Validation Results:
Val Age Accuracy: 0.6321
Val Gender Accuracy: 0.8557
Val Emotion Accuracy: 0.6570


#### 8. Phase 2 - Fine-tune top EfficientNetB0 blocks

* Freezing BN layers is standard to keep their statistics stable during fine‑tuning.
* Lower LR prevents destroying pretrained weights while still adapting to your tasks.

In [27]:
# Choose how much of EfficientNetB0 to unfreeze:
# Here we unfreeze the last N layers (you can tune this).
fine_tune_at = int(len(base_model.layers) * 0.7)  # unfreeze top 30% of layers

for layer in base_model.layers[fine_tune_at:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Re-compile with lower learning rate for fine-tuning
multi_task_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)

history_finetune = multi_task_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINETUNE,
)

Epoch 1/10
138/910 ━━━━━━━━━━━━━━━━━━━━ 5:41 442ms/step - age_acc: 0.6865 - age_loss: 0.7633 - emotion_acc: 0.7030 - emotion_loss: 0.8578 - gender_acc: 0.8717 - gender_loss: 0.2923 - loss: 1.9134

Corrupt JPEG data: premature end of data segment


310/910 ━━━━━━━━━━━━━━━━━━━━ 6:57 695ms/step - age_acc: 0.6958 - age_loss: 0.7536 - emotion_acc: 0.7044 - emotion_loss: 0.8087 - gender_acc: 0.8742 - gender_loss: 0.2872 - loss: 1.8495

Corrupt JPEG data: premature end of data segment


910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - age_acc: 0.7077 - age_loss: 0.7296 - emotion_acc: 0.7197 - emotion_loss: 0.7196 - gender_acc: 0.8775 - gender_loss: 0.2698 - loss: 1.7191

...

310/910 ━━━━━━━━━━━━━━━━━━━━ 7:10 717ms/step - age_acc: 0.8689 - age_loss: 0.3562 - emotion_acc: 0.9488 - emotion_loss: 0.1454 - gender_acc: 0.9363 - gender_loss: 0.1562 - loss: 0.6578

Corrupt JPEG data: premature end of data segment


910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 714ms/step - age_acc: 0.8671 - age_loss: 0.3496 - emotion_acc: 0.9537 - emotion_loss: 0.1292 - gender_acc: 0.9356 - gender_loss: 0.1467 - loss: 0.6255

2026-02-22 22:16:06.756383: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


910/910 ━━━━━━━━━━━━━━━━━━━━ 772s 846ms/step - age_acc: 0.8671 - age_loss: 0.3502 - emotion_acc: 0.9538 - emotion_loss: 0.1292 - gender_acc: 0.9356 - gender_loss: 0.1467 - loss: 0.6261 - val_age_acc: 0.6347 - val_age_loss: 0.9664 - val_emotion_acc: 0.7418 - val_emotion_loss: 0.4065 - val_gender_acc: 0.8680 - val_gender_loss: 0.1787 - val_loss: 1.5516


#### 9. Simple inference helper

In [28]:
# Inference helper on a single image

age_label_map = {i: i for i in range(NUM_AGE_CLASSES)}  # or bins
gender_label_map = {0: "male", 1: "female"}             # adjust if needed
emotion_label_map = {i: f"class_{i}" for i in range(NUM_EMOTION_CLASSES)}  # replace later

def predict_on_image(img_path):
    img = load_and_preprocess(img_path)
    img = tf.expand_dims(img, axis=0)

    age_logits, gender_logits, emotion_logits = model(img, training=False)

    age_pred = tf.argmax(age_logits, axis=-1).numpy()[0]
    gender_pred = tf.argmax(gender_logits, axis=-1).numpy()[0]
    emotion_pred = tf.argmax(emotion_logits, axis=-1).numpy()[0]

    print("Pred age bin:", age_label_map[age_pred])
    print("Pred gender:", gender_label_map[gender_pred])
    print("Pred emotion:", emotion_label_map[emotion_pred])

# Example:
example_path = train_df.iloc[0]["image_path"]
# print("Example image path:", example_path)
# printing the image object to verify it's correct
img = load_and_preprocess(example_path)
print("Loaded image shape:", img.shape)
predict_on_image(example_path)

Loaded image shape: (224, 224, 3)
Pred age bin: 3
Pred gender: male
Pred emotion: class_4


In [29]:
# Phase 1 accuracies
print("Phase 1 - Final Epoch:")
print(f"Age Accuracy: {history_heads.history['age_acc'][-1]:.4f}")
print(f"Gender Accuracy: {history_heads.history['gender_acc'][-1]:.4f}")
print(f"Emotion Accuracy: {history_heads.history['emotion_acc'][-1]:.4f}")
print(f"Val Age Accuracy: {history_heads.history['val_age_acc'][-1]:.4f}")
print(f"Val Gender Accuracy: {history_heads.history['val_gender_acc'][-1]:.4f}")
print(f"Val Emotion Accuracy: {history_heads.history['val_emotion_acc'][-1]:.4f}")

# Phase 2 accuracies
print("\nPhase 2 - Final Epoch:")
print(f"Age Accuracy: {history_finetune.history['age_acc'][-1]:.4f}")
print(f"Gender Accuracy: {history_finetune.history['gender_acc'][-1]:.4f}")
print(f"Emotion Accuracy: {history_finetune.history['emotion_acc'][-1]:.4f}")
print(f"Val Age Accuracy: {history_finetune.history['val_age_acc'][-1]:.4f}")
print(f"Val Gender Accuracy: {history_finetune.history['val_gender_acc'][-1]:.4f}")
print(f"Val Emotion Accuracy: {history_finetune.history['val_emotion_acc'][-1]:.4f}")

Phase 1 - Final Epoch:
Age Accuracy: 0.7015
Gender Accuracy: 0.8772
Emotion Accuracy: 0.7240
Val Age Accuracy: 0.6321
Val Gender Accuracy: 0.8557
Val Emotion Accuracy: 0.6570

Phase 2 - Final Epoch:
Age Accuracy: 0.8663
Gender Accuracy: 0.9369
Emotion Accuracy: 0.9582
Val Age Accuracy: 0.6347
Val Gender Accuracy: 0.8680
Val Emotion Accuracy: 0.7418


#### Saving the Model for future 

In [33]:
# Saving the model
model.save("facial_multitask_model.keras")